DeepLearningCourse vol.5
# chainer CNN チュートリアル with MNIST

In [2]:
# 必要なモジュールのインポート

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### CNNの定義
今回は以下のようなCNNを定義します。
![](https://s3.amazonaws.com/ai-standard/pic-d5-1.png)

In [3]:
class CNN(chainer.Chain):

    def __init__(self, train=True):
        super(CNN, self).__init__(
            conv1=L.Convolution2D(1, 32, 5),
            conv2=L.Convolution2D(32, 64, 5),
            l1=L.Linear(1024, 10),
        )
        self.train = train

    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        return self.l1(h)

モデルのCNN以外は以前のコードとほぼ同じです。

In [4]:
# ミニバッチのサイズを指定
batchsize = 100

# 学習の繰り返し回数（epoch）を指定
epoch = 20

# 訓練のためのニューラルネットワークをセットアップする
model = L.Classifier(CNN())

In [5]:
# MNISTのデータセットを読み込む
# ndim に 3 を与えると，shape が (1, 28, 28) の形式で取得できます。
train, test = chainer.datasets.get_mnist(ndim=3)

# データセットをイテレータの形式に変換する
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

# 最適化の方法を定義
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

# updaterにiteratorとoptimizerを渡す
updater = training.StandardUpdater(train_iter, optimizer)

# trainerにupdaterを渡す
trainer = training.Trainer(updater, (epoch, 'epoch'))

# epochごとに学習後の精度を評価する
trainer.extend(extensions.Evaluator(test_iter, model))

# epochごとにlogを出力する
trainer.extend(extensions.LogReport())

# logに出力する情報を指定する
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss',
                                       'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

# 構成したtrainerをもとに､学習を実行する
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           0.165093    0.0487811             0.95185        0.9836                    231.554       
2           0.0489182   0.0323509             0.985          0.9899                    441.367       
3           0.0351906   0.0254092             0.988917       0.9921                    655.832       
4           0.0273997   0.0416213             0.991383       0.9866                    844.852       
5           0.0217551   0.0288202             0.993033       0.9908                    1021.96       
6           0.017318    0.0278816             0.994517       0.9908                    1197.7        
7           0.0142615   0.0292009             0.995583       0.992                     1389.35       
8           0.0110867   0.0281081             0.996467       0.9915                    1593.46       
9           0.0107946   0.0264123             0.9964         0.9918                 

### 精度比較
![](https://s3.amazonaws.com/ai-standard/pic-d5-4.png)

### loss比較
![](https://s3.amazonaws.com/ai-standard/pic-d5-5.png)

以上から、単純なCNNでもニューラルネットワークと比べると、精度・lossが改善されていることが分かりました。